## Classification Unimodal Metric Analysis

The goal of this notebook is to asses the accuracy of the classification assignment in the ML algorithm with regards to different travel modes.

## Set up the dependencies

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev
import emeval.viz.geojson as ezgj

In [ ]:
# Analytics results
import emeval.metrics.segmentation as ems

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format
import arrow
import numpy as np

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# For maps
import folium
import branca.element as bre

In [ ]:
# For easier debugging while working on modules
import importlib

In [ ]:
import arrow

## The spec

The spec defines what experiments were done, and over which time ranges. Once the experiment is complete, most of the structure is read back from the data, but we use the spec to validate that it all worked correctly. The spec also contains the ground truth for the legs. Here, we use all three specs provided as per the documentation.

## Import from Server
This function will import the phone view data from the server and write the phone view data to a `.pkl` file if optioned. This function will return a list of spec details and phone views for the specified trips.

### Emperical Time Test:
The following experiment was run on this function with all default arguments

#### Spec:
* Mac OSX 12.3.1 
* 2.8 GHz Quad-Core Intel Core i7 CPU 
* 16 GB 1600 MHz DDR3 RAM

#### Time (using %%time): 
* `CPU times: user 47.9 s, sys: 5.67 s, total: 53.5 s`
* `Wall time: 19min 11s`

In [ ]:
def import_sd_and_pv_from_server(trips  = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"], 
                                 AUTHOR_EMAIL  = "shankari@eecs.berkeley.edu", 
                                 DATASTORE_LOC = "http://localhost:8080", 
                                 pkl_file_name = None):
    sd_l = []
    pv_l = []
    for trip in trips:
        sd = eisd.ServerSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, trip)
        pv = eipv.PhoneView(sd)
        sd_l.append(sd)
        pv_l.append(pv)
    if pkl_file_name:
        import pickle
        with open(pkl_file_name, 'wb') as outp:
            for pv in pv_l:
                pickle.dump(pv, outp, pickle.HIGHEST_PROTOCOL)
    return sd_l, pv_l

## Import from .pkl (Recomended)

### Emperical Time Test:
The following experiment was run on this function with all default arguments

#### Spec:
* Mac OSX 12.3.1 
* 2.8 GHz Quad-Core Intel Core i7 CPU 
* 16 GB 1600 MHz DDR3 RAM

#### Time (using %%time): 
* `CPU times: user 12.4 s, sys: 2.41 s, total: 14.8 s`
* `Wall time: 15.9 s`

In [ ]:
def import_pv_from_pkl(pkl_file_name, 
                       trips = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"]):
    import pickle
    pv_l = []
    with open('pv.pkl', 'rb') as inp:
        for trip in trips:
            pv_l.append(pickle.load(inp))
    return pv_l

In [ ]:
(pv_la, pv_sj, pv_ucb) = import_pv_from_pkl('pv.pkl')

Get the sensed section data for each trip

In [ ]:
%%capture
ems.fill_sensed_section_ranges(pv_la)
ems.fill_sensed_section_ranges(pv_sj)
ems.fill_sensed_section_ranges(pv_ucb)

## Get Sensed Mode Entries

This function is the bread and butter of this notebook. The most important metric in this function is `matching_pct`, witch is retured as part of a dictionary from the call to `ems.get_mode_check_results(section, section_gt_leg, matching_section_map)`. `matching_pct` is defined by the following sequence of code:
```python
matching_sections = [s for s in sensed_section_range if s["mode"] == base_mode]
matching_ts = sum([(s["end_ts"] - s["start_ts"]) for s in matching_sections])
gt_duration = (segment["end_ts"] - segment["start_ts"])
matching_pct = matching_ts / gt_duration
```
which in turn is directly the metric described in [UCB/EECS-2019-180](http://www2.eecs.berkeley.edu/Pubs/TechRpts/2019/EECS-2019-180.html) pg. 129

$$
a_s = \mathrm{ \sum \limits_{ss_{gts} \in SS_{gts},~ss_{gts}.label=gts.label} \frac{|ss_{gts}.end\_ts - ss_{gts}.start\_ts|}{|{gts}.end\_ts - {gts}.start\_ts|} } ~ \forall gts \in GTS
$$

as means to caputre error in sensed mode assignment. As this metric describes the time spent in matching modes, a resulting $a_s$ as close to $1$ is preferable.

In [ ]:
def get_sensed_mode_entries(pv):
    sensed_mode_entry_list = []
    for phone_os, phone_map in pv.map().items():
        for phone_label, phone_detail_map in phone_map.items():
            if "control" in phone_detail_map["role"]:
                print("Ignoring %s phone %s since they are always on" % (phone_detail_map["role"], phone_label))
                continue
            # this spec does not have any calibration ranges, but evaluation ranges are actually cooler
            for r in phone_detail_map["evaluation_ranges"]:
                for tr in r["evaluation_trip_ranges"]:
                    matching_section_map = ems.find_matching_segments(tr["evaluation_section_ranges"], 
                                                                      "trip_id", tr["sensed_section_ranges"])
                    for section in tr["evaluation_section_ranges"]:
                        section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr['trip_id_base'],
                                                                                  section['trip_id_base'],
                                                                                  tr['start_ts'],
                                                                                  tr['end_ts'])
                        if section_gt_leg["type"] == "WAITING":
                            print("Skipping WAITING section %s %s with potential partway transitions" %
                                  (tr["trip_id"], section["trip_id"]))
                            continue
                        # this calulcates the metric for the mode
                        result = ems.get_mode_check_results(section, section_gt_leg, matching_section_map)
                        sensed_mode_entry = {"phone_os": phone_os, 
                                             "phone_label": phone_label,
                                             "timeline": pv.spec_details.curr_spec["id"],
                                             "range_id": r["trip_id"],
                                             "run": r["trip_run"], 
                                             "duration": r["duration"],
                                             "role": r["eval_role_base"],
                                             "section_count": len(tr["sensed_section_ranges"]),
                                             "trip_id": tr["trip_id"],
                                             "section_id": section["trip_id"]}
                        sensed_mode_entry.update(result)
                        sensed_mode_entry_list.append(sensed_mode_entry)

    return sensed_mode_entry_list

In [ ]:
%%capture
sensed_mode_entries_list = []
for pv in [pv_la, pv_sj, pv_ucb]:
    sensed_mode_entries_list.extend(get_sensed_mode_entries(pv))
sensed_mode_df = pd.DataFrame(sensed_mode_entries_list)

## Data Analysis

For the purpose of this study, we analyze the `matching_pct` metric with regards to each sensed mode of travel, in hopes to inform a future error bound on sensed mode data.

# Plot Mode vs. Matching Percentage data

#### First, we plot just the base modes

In [ ]:
ifig, ax_array = plt.subplots(nrows=1, ncols=3, figsize=(24,12), dpi=300, sharex=False, sharey=False)
for i, mode in enumerate(sensed_mode_df.gt_base_mode.unique()):
    sensed_mode_df.query(f"gt_base_mode == '{mode}'").boxplot(ax=ax_array[i], 
                                                              column=['matching_pct'], 
                                                              by=["gt_base_mode"])

#### Now we plot all the available modes

In [ ]:
ifig, ax_array = plt.subplots(nrows=2, ncols=4, figsize=(24,12), dpi=300, sharex=False, sharey=False)
for i, mode in enumerate(sensed_mode_df.gt_mode.unique()):
    sensed_mode_df.query(f"gt_mode == '{mode}'").boxplot(ax=ax_array.flatten()[i], 
                                                              column=['matching_pct'], 
                                                              by=["gt_mode"])

#### Finaly, we see if any indavidual modes stick out amung the respective base modes

In [ ]:
%%capture --no-display
ifig, ax_array = plt.subplots(nrows=1, ncols=3, figsize=(24,12), dpi=300, sharex=False, sharey=False)
for i, mode in enumerate(sensed_mode_df.gt_base_mode.unique()):
    sensed_mode_df.query(f"gt_base_mode == '{mode}'").boxplot(ax=ax_array[i], 
                                                              column=['matching_pct'], 
                                                              by=["gt_mode"])
ifig.suptitle("Boxplot grouped by gt_mode distributed by gt_base_mode")

## And for thoes who care about the numbers

Formating inspired by this stack post: https://stackoverflow.com/a/50899244

In [ ]:
from IPython.display import display_html 
display_list = []
for i, mode in enumerate(sensed_mode_df.gt_base_mode.unique()):
    df = sensed_mode_df.query(f"gt_base_mode == '{mode}'").describe()
    df_styler = df.style.set_table_attributes("style='display:inline'").set_caption(f"BASE MODE: {mode}")    
    display_list.append(df_styler)
display_html(''.join([display_list[k]._repr_html_() for k in range(i + 1)]), raw=True)

In [ ]:
display_list = []
for i, mode in enumerate(sensed_mode_df.gt_mode.unique()):
    df = sensed_mode_df.query(f"gt_mode == '{mode}'").describe()
    df_styler = df.style.set_table_attributes("style='display:inline'").set_caption(f"MODE: {mode}")    
    display_list.append(df_styler)
display_html(''.join([display_list[k]._repr_html_() for k in range(i + 1)]), raw=True)

`TODO:` Sensed mode distribution with error bars on envaluation trip ranges

## Get the Sensed Section Modes
takes in a phone view and a run range - either "evaluation_ranges" or "sensed_ranges" - defaults to "evaluation_ranges" and returns the sensed modes.

In [ ]:
def get_sensed_section_modes(pv):
    modes = []
    for phone_os, phone_map in pv.map().items():
        for phone_label, phone_detail_map in phone_map.items():
            if "control" in phone_detail_map["role"]:
                print("Ignoring %s phone %s since they are always on" % (phone_detail_map["role"], phone_label))
                continue
            # this spec does not have any calibration ranges, but evaluation ranges are actually cooler
            for r in phone_detail_map["evaluation_ranges"]:
                for tr in r["evaluation_trip_ranges"]:
                    for sensed_section in tr["sensed_section_ranges"]:
                        sensed_mode_entry = {"phone_os": phone_os, 
                                             "phone_label": phone_label,
                                             "timeline": pv.spec_details.curr_spec["id"],
                                             "range_id": r["trip_id"],
                                             "run": r["trip_run"], 
                                             "duration": r["duration"],
                                             "role": r["eval_role_base"],
                                             "section_count": len(tr["sensed_section_ranges"]),
                                             "trip_id": tr["trip_id"],
                                             "start_ts" : sensed_section["start_ts"], 
                                             "end_ts" : sensed_section["end_ts"],
                                             "sensed_mode": sensed_section["mode"]}
                        modes.append(sensed_mode_entry)
    return modes

In [ ]:
%%capture
eval_trip_sensed_modes = []
for pv in [pv_la, pv_sj, pv_ucb]:
    eval_trip_sensed_modes.extend(get_sensed_section_modes(pv))
eval_trip_sensed_modes_df = pd.DataFrame(eval_trip_sensed_modes)

In [ ]:
eval_trip_sensed_modes_df.sensed_mode.unique()

In [ ]:
ifig, ax_array = plt.subplots(nrows=1, ncols=2, figsize=(12,6), dpi=300, sharex=False, sharey=False)

eval_trip_sensed_modes_df.set_index('sensed_mode').groupby(['sensed_mode']).sum().plot.pie(y='duration', ax=ax_array[0], legend=False)
eval_trip_sensed_modes_df.set_index('sensed_mode').groupby(['sensed_mode']).sum().plot.bar(y='duration', ax=ax_array[1])

## Get sensed timeline

```python
def get_trip_ss_and_gts_timeline():
    """
    Get the sensed and ground truth timeline for each evaluation trip range for a given phone view.
    
    ----------
    Parameters
    ----------
    arg1: phone view
        A phone view to recieve timelines for.

    -------
    Returns
    -------
    list
        A list of trips for each phone view. Each trip has two entries, the sensed mode timeline and the ground truth timeline for the corresponding evaluation trip range.
    """
    ...
```

In [ ]:
def get_trip_ss_and_gts_timeline(pv):
    trips = []
    for phone_os, phone_map in pv.map().items():
        for phone_label, phone_detail_map in phone_map.items():
            if "control" in phone_detail_map["role"]:
#                 print("Ignoring %s phone %s since they are always on" % (phone_detail_map["role"], phone_label))
                continue
            # this spec does not have any calibration ranges, but evaluation ranges are actually cooler
            for r in phone_detail_map["evaluation_ranges"]:
                for tr in r["evaluation_trip_ranges"]:
                    tr_ss  = []
                    tr_gts = []
                    for ss in tr["sensed_section_ranges"]:
                        tr_ss.append(ss)
                    for section in tr["evaluation_section_ranges"]:
                        section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr['trip_id_base'],
                                                                                  section['trip_id_base'],
                                                                                  tr['start_ts'],
                                                                                  tr['end_ts'])
                        if section_gt_leg["type"] == "WAITING":
#                             print("Skipping WAITING section %s %s with potential partway transitions" %
#                                   (tr["trip_id"], section["trip_id"]))
                            continue
                        # this calulcates the metric for the mode

                        ## and now we have the gt mode!
                        gts = {'start_ts': section['start_ts'], 
                               'end_ts': section['end_ts'], 
                               'mode': section_gt_leg['mode']}
                        tr_gts.append(gts)
                # now, we build a timeline for each trip
                trip = tr.copy()
                trip['ss_timeline']  = tr_ss
                trip['gts_timeline'] = tr_gts
                trips.append(trip)
    return trips

## Binary Classification
```python
def get_binary_class(pv_l=[pv_la, pv_sj, pv_ucb]):
    """
    This function computes binary classifications for a given set of trips.
    Using one unit of duration as our base unit, we calculate the following classifications:
        * True Positive
            + A true positive is when we sense that we are in a mode and we are in that mode.
        * False Positive
            + A false positive is when we sense that we are in a mode that we are not it.
        * False Negative
            + A false negative is when we are in a mode, but we do not sense being in that mode.
     
     Note that we compute the binary classifications for each sensed mode, but we combine the 'WALKING' and 'RUNNING' modes.
     Additionally, note that we have use the ground truth base mode when determining hits and misses. 
    
    ----------
    Parameters
    ----------
    arg1: phone view list
        A list of phone views to retrieve sensed and ground truth timelines for. Defaults to phone views from the LA, SJ, and UCB trips.
    
    -------
    Returns
    -------
    list
        A list with the following entries
            [0] A dictionary of true positives with sensed modes as keys and TP hits has values.
            [1] A dictionary of false positives with sensed modes as keys and FP hits has values.
            [2] A dictionary of false negatives with sensed modes as keys and FN hits has values.
    
    """
    ...
```

In [ ]:
def get_binary_class(pv_l=[pv_la, pv_sj, pv_ucb]):
    BASE_MODE = {"WALKING": "WALKING",
        "BICYCLING": "CYCLING",
        "ESCOOTER": "CYCLING",
        "BUS": "AUTOMOTIVE",
        "TRAIN": "AUTOMOTIVE",
        "LIGHT_RAIL": "AUTOMOTIVE",
        "SUBWAY": "AUTOMOTIVE",
        "CAR": "AUTOMOTIVE"}
    trips = []
    for pv in pv_l:
        trips.extend(get_trip_ss_and_gts_timeline(pv))
    TP = {'INVALID' : 0}
    FP = {}
    FN = {}
    ## loop though all of the trips
    for trip in trips:
        ## when we sense a mode. Used for TP and FP
        for ss in trip['sensed_section_ranges']:
            # sensed duration := TP + FP
            ss_dur  = ss['end_ts'] - ss['start_ts']
            time_in_gt_modes = {}
            gts_dur = 0
            for gts in trip['gts_timeline']:
                # if the gts starts before the ss ends and ends after the ss starts
                if gts['start_ts'] <= ss['end_ts'] and gts['end_ts'] >= ss['start_ts']:
                    dur     = min(ss['end_ts'], gts['end_ts']) - max(ss['start_ts'], gts['start_ts'])
                    gts_dur += dur
                    if BASE_MODE[gts['mode']] == ss['mode']:
                        # we sense that we are in a mode, and we are
                        TP[ss['mode']] = time_in_gt_modes.setdefault(ss['mode'], 0) + dur
                    else:
                        # we sense that we are in a mode, and we are not
                        FP[ss['mode']] = time_in_gt_modes.setdefault(ss['mode'], 0) + dur
            ## Take care of when we have no gts, which is a FP as we sense we are in a mode, but in fact their is no gt, so we are not
            leftover = ss_dur - gts_dur
            assert leftover >= 0
            FP[ss['mode']] = time_in_gt_modes.setdefault(ss['mode'], 0) + leftover # FIXME: this gives mode error
        ## Now we check the ground truth timeline, for FN
        for gts in trip['gts_timeline']:
            gts_dur = gts['end_ts'] - gts['start_ts']
            ss_dur = 0
            for ss in trip['sensed_section_ranges']:
                # if the ss starts before gts ends and ss ends after the gts starts
                if ss['start_ts'] <= gts['end_ts'] and ss['end_ts'] >= gts['start_ts']:
                    dur     = min(ss['end_ts'], gts['end_ts']) - max(ss['start_ts'], gts['start_ts'])
                    ss_dur += dur
                    if BASE_MODE[gts['mode']] != ss['mode']:
                        # we sense that we are not in a mode, but we are
                        FN[ss['mode']] = time_in_gt_modes.setdefault(ss['mode'], 0) + dur
            leftover = gts_dur - ss_dur
            assert leftover >= 0
            # we sense that we are not in a mode, but we are
            FN[ss['mode']] = time_in_gt_modes.setdefault(ss['mode'], 0) + leftover
    ## add together running and walking for TP and FN and remove the running key
    for BC in FP,FN:
        BC['WALKING'] += BC['RUNNING']
        BC.pop('RUNNING')
    return [TP, FP, FN]

# $F_\beta$ score
$$
F_\beta = \frac {(1 + \beta^2) \cdot \mathrm{true\ positive} }{(1 + \beta^2) \cdot \mathrm{true\ positive} + \beta^2 \cdot \mathrm{false\ negative} + \mathrm{false\ positive}}
$$

```python
def get_F_score(pv_l=[pv_la, pv_sj, pv_ucb], beta=1):
    """
    This function calculates the F score
    $$
    F_\beta = \frac {(1 + \beta^2) \cdot \mathrm{true\ positive} }{(1 + \beta^2) \cdot \mathrm{true\ positive} + \beta^2 \cdot \mathrm{false\ negative} + \mathrm{false\ positive}}
    $$
    based off data from a given set of phone views. Calls the get binary classification function
    
    ----------
    Parameters
    ----------
    arg1: phone view list
        list of phone views to collect data from. Defaults to the LA, SJ, and UCB trip phone views
    arg2: int
        The beta value in which to use in the $F_\beta$ score. Defaults to 1.
        
    -------
    Returns
    -------
    dict:
        A dictionary with sensed modes as the keys and the corresponding of $F_\beta$ scores as the values.
    
    """
    ...
```

In [ ]:
def get_F_score(pv_l=[pv_la, pv_sj, pv_ucb], beta=1):
    F_score = {}
    (TP, FP, FN) = get_binary_class(pv_l)
    for mode in FP.keys():
        numerator   = (1 + beta**2) * TP[mode]
        denominator = (1+beta**2) * TP[mode] + beta**2*FN[mode] + FP[mode]
        F_score[mode] = (numerator)/(denominator)
    return F_score

In [ ]:
get_F_score()

## Algorithm

```
for each trip:
    for each sensed section in trip:
        for each ground truth section in trip:
            if the sensed section ocurrs at any point in a ground truth section:
                record the ground truth mode and the durration spent in that mode
        record the difference between sensed section duration and the time spend in each ground truth mode
        divide time spent in ground truth modes for all modes
        ledger the sensed mode, as well as the ratio of time spend in each groud truth mode
group all sensed modes and determine the percentage of time spent in each ground truth mode for each sensed mode

```

# Confusion Matrix
For every sensed section, lets find ground truth associated with that time frame, and we find the ratio of ground truth modes for that given time frame, and compare with the sensed mode.

In [ ]:
def get_ss_mode_v_gt_mode(pv_l=[pv_la, pv_sj, pv_ucb]):
    trips = []
    for pv in pv_l:
        trips.extend(get_trip_ss_and_gts_timeline(pv))
    ss_mode_v_gt_mode_l = []
    for trip in trips:
        probabilities = {}
        for ss in trip['sensed_section_ranges']:
            ss_dur  = ss['end_ts'] - ss['start_ts']
            time_in_gt_modes = {}
            for i, gts in enumerate(trip['gts_timeline']): # TODO: optimize here
                if gts['end_ts'] >= ss['start_ts'] and gts['start_ts'] <= ss['end_ts']:
                    dur     = min(ss['end_ts'], gts['end_ts']) - max(ss['start_ts'], gts['start_ts'])
                    time_in_gt_modes[gts['mode']] = time_in_gt_modes.setdefault(gts['mode'], 0) + dur
            time_in_gt_modes['NO GT'] = max(ss_dur - sum(time_in_gt_modes.values(), 0.0),0) #TODO: make sure this is right
            for mode in time_in_gt_modes.keys():
                time_in_gt_modes[mode] /= ss_dur
            # TODO: assert sum of all rows == 1 +- epsilon
            time_in_gt_modes['sensed_mode'] = ss['mode']
            ss_mode_v_gt_mode_l.append(time_in_gt_modes)
    return ss_mode_v_gt_mode_l

In [ ]:
ss_mode_v_gt_mode_df = pd.DataFrame(get_ss_mode_v_gt_mode([pv_la, pv_sj, pv_ucb])).groupby('sensed_mode').sum()
ss_mode_v_gt_mode_df.div(ss_mode_v_gt_mode_df.sum(axis=1), axis=0).mul(100)

# Now for the Analyzed Data

To analyze the data, we run it throught the e-mission pipeline, maming sure to run `source setup/activate.sh` to activate our conda environemnt, and then we run `./e-mission-py.bash bin/reset_pipeline.py -a` to reset the pipeline. Finally, we run the following python script in the e-mission directory
```python
import logging
from logging import config
import emission.core.get_database as edb
import emission.pipeline.intake_stage as epi
users = list(edb.get_uuid_db().find())
uuid_list = []
for user in users:
    uuid_list.append(user['uuid'])
# I think the following process number should work. Not to be cynical, but it is only used for logging, so I don't care too much
epi.run_intake_pipeline(str(len(uuid_list)), uuid_list)

```


Then we load in the anlyized data with `emeval.analysed.phone_view.create_analysed_view()`

p.s. also, we can totaly save it as a .pkl file localy for speed :)

In [ ]:
import emeval.analysed.phone_view as eapv

In [ ]:
av_la = eapv.create_analysed_view(pv_la, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")

In [ ]:
av_sj = eapv.create_analysed_view(pv_sj, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")

In [ ]:
av_ucb = eapv.create_analysed_view(pv_ucb, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")

In [ ]:
ss_mode_v_gt_mode_df = pd.DataFrame(get_ss_mode_v_gt_mode([av_la, av_sj, av_ucb])).groupby('sensed_mode').sum()
ss_mode_v_gt_mode_df.div(ss_mode_v_gt_mode_df.sum(axis=1), axis=0).mul(100)